In [19]:
import pandas as pd
from llama_index.core import Document, SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.settings import Settings
from llama_index.core.chat_engine import CondenseQuestionChatEngine
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.llms.llama_cpp import LlamaCPP
from llama_cpp import Llama
from sentence_transformers import SentenceTransformer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding    
import faiss
import os
import json
import re
from Bio import Entrez
import time
import xmltodict
from pathlib import Path

In [20]:
training_dataset_path = r"E:\RAG_Models\BioASQ-training13b\training13b.json"
abstract_cache = Path(r".\Abstracts")
out_csv = "bioasq_rag_predictions.csv"
email = "an80@illinois.edu"

In [21]:
abstract_cache.mkdir(exist_ok=True)

In [14]:
Entrez.email = email

In [6]:
with open(training_dataset_path,"r") as file:
    json_data = json.load(file)

In [7]:
document_urls = []
questions = []
ground_truth = []
for data in json_data['questions'][:100]:
   document_urls.extend(data.get('documents'))
   questions.extend(data.get("body"))
   ground_truth.extend(data.get("ideal_answer"))


In [9]:
print(f'Checking for duplicate documents. Original number of documents: {len(document_urls)}')
document_urls = list(set(document_urls))
print(f'Number of documents after filtering: {len(document_urls)}')

Checking for duplicate documents. Original number of documents: 1142
Number of documents after filtering: 1136


In [10]:
PMID_RE = re.compile(r"/pubmed/(\d+)")

In [11]:
def pmid_from_url(url: str) -> str:
    m = PMID_RE.search(url)
    return m.group(1) if m else None

In [12]:
pmids = [pmid_from_url(doc) for doc in document_urls]

In [23]:
def batch_fetch_pmids(pmids, batch = 200):
    """Return {pmid: {'title','abstract','year'}}; caches and skips empty abstracts."""
    out = {}
    for i in range(0, len(pmids), batch):
        chunk = pmids[i:i + batch]
        need = [p for p in chunk if not (abstract_cache / f"{p}.json").exists()]
        if need:
            raw_xml = Entrez.efetch(
                db="pubmed",
                id=",".join(need),
                rettype="abstract",
                retmode="xml"
            ).read()
            xml = xmltodict.parse(raw_xml)
            for art in xml["PubmedArticleSet"]["PubmedArticle"]:
                pmid = art["MedlineCitation"]["PMID"]["#text"]
                art_info = art["MedlineCitation"]["Article"]
                title = art_info.get("ArticleTitle", "")

                # ---- robust abstract extraction ----
                abs_raw = art_info.get("Abstract", {}).get("AbstractText", [])
                if isinstance(abs_raw, list):
                    parts = [x.get("#text", "") if isinstance(x, dict) else str(x) for x in abs_raw]
                    abstract = " ".join(parts).strip()
                else:
                    abstract = str(abs_raw).strip()

                if not abstract:   # skip if empty
                    continue

                year = art_info["Journal"]["JournalIssue"]["PubDate"].get("Year", "Unknown")

                meta = {"title": title, "abstract": abstract, "year": year}
                (abstract_cache / f"{pmid}.json").write_text(json.dumps(meta))

        # load all cached (existing + newly saved)
        for pmid in chunk:
            fp = abstract_cache / f"{pmid}.json"
            if fp.exists():
                meta = json.loads(fp.read_text())
                if meta.get("abstract", "").strip():
                    out[pmid] = meta
        time.sleep(0.4)  # politeness
    return out

In [24]:
data_map = batch_fetch_pmids(pmids)